In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bertopic-0.14.1-py2.py3-none-any.whl (120 kB)
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 1

In [2]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/Amazon reviews/sample_data_for_topic_modelling.csv')

In [6]:
data.shape

(25129, 23)

In [7]:
data.head()

,Unnamed: 0.1,Unnamed: 0,ID,Subject,Description,sub_words_count,sub_char_count,desc_words_count,desc_char_count,Cleaned_Desc,...,desc_score,desc_sentiment,Date/Time Opened,Case Category,Category,Sub Category,Case Close Reason,Product Type,Origin,Top Management Flag
0,30,30,31,Planet App Feedback,Very poor customer service no response from yo...,3,19,21,143,Very poor customer service no response from yo...,...,0.9392,negative,2022-12-31T19:12:39.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,Two Wheeler,Mobile app,No
1,131,131,132,L&T Financial Services - General Enquiry,Two wheeler loan still not update\n\n-- \nDisc...,6,40,176,1115,Two wheeler loan still not update,...,0.6240,negative,2022-12-31T19:49:36.000+0000,Query,Payment & Refund,Payment Confirmation,Refer to this case id :17768143,Two Wheeler,Email,No
2,167,167,168,Loan not clear,NAME - NARMADA DAS\nMOBILE NO - 9678183958\nLO...,3,14,205,1265,NAME NARMADA DAS MOBILE NO LOAN NO C I have pa...,...,0.6760,negative,2022-12-31T20:07:44.000+0000,Query,Account Closure,Account Status is Active,"Dear Customer,\r\n\r\nThank you for writing to...",Consumer Loan,Email,No
3,226,226,227,L&T Financial Services - General Enquiry,"Dear Team,\n\n\nI have already paid the emi of...",6,40,214,1355,Dear Team I have already paid the emi of rd Ja...,...,0.4921,negative,2022-12-31T20:51:17.000+0000,Query,General Enquiry,LTFS Related,We acknowledge the receipt of your email and a...,Others,Email,No
4,292,292,293,LAN C022122501011708,"Dear Sir/Madam,\n\nI have taken loan from you ...",2,20,258,1571,Dear Sir Madam I have taken loan from you but ...,...,0.7081,negative,2022-12-31T21:58:37.000+0000,Query,Insurance,Insurance Related,We regret to inform you that we are unable to ...,Consumer Loan,Email,No


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25129 entries, 0 to 25128
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0.1              25129 non-null  int64  
 1   Unnamed: 0                25129 non-null  int64  
 2   ID                        25129 non-null  int64  
 3   Subject                   24794 non-null  object 
 4   Description               25129 non-null  object 
 5   sub_words_count           25129 non-null  int64  
 6   sub_char_count            25129 non-null  int64  
 7   desc_words_count          25129 non-null  int64  
 8   desc_char_count           25129 non-null  int64  
 9   Cleaned_Desc              25129 non-null  object 
 10  cleaned_desc_words_count  25129 non-null  int64  
 11  subject_score             25129 non-null  float64
 12  subject_sentiment         25129 non-null  object 
 13  desc_score                25129 non-null  float64
 14  desc_s

In [9]:
import re, string

In [10]:
def preprocess(CleanText):
    CleanText = CleanText.str.replace("(<br/>)", "")
    CleanText = CleanText.str.replace('(<a).*(>).*(</a>)', '')
    CleanText = CleanText.str.replace('(&amp)', '')
    CleanText = CleanText.str.replace('(&gt)', '')
    CleanText = CleanText.str.replace('(&lt)', '')
    CleanText = CleanText.str.replace('(\xa0)', ' ')
    # CleanText = " ".str.join(CleanText.str.split())  
    return CleanText
data['Cleaned_Desc'] = preprocess(data['Cleaned_Desc'])

In [11]:
type(data['Cleaned_Desc'])

pandas.core.series.Series

In [12]:
# create model 
 
model = BERTopic(verbose=True)
 
#convert to list 
docs = data.Cleaned_Desc.to_list()
 
topics, probabilities = model.fit_transform(docs)

Batches:   0%|          | 0/786 [00:00<?, ?it/s]

2023-05-23 09:06:54,483 - BERTopic - Transformed documents to Embeddings
2023-05-23 09:08:33,540 - BERTopic - Reduced dimensionality
2023-05-23 09:08:40,861 - BERTopic - Clustered reduced embeddings


In [22]:
print(len(model.get_topic(4)))

10


In [13]:
model.get_topic_freq()

,Topic,Count
0,-1,3261
1,0,825
2,1,823
3,2,800
4,3,440
...,...,...
533,532,10
534,533,10
535,534,10
536,535,10


In [14]:
def output(x):
  return model.get_topic(x)

In [15]:
for j in range(515):
  print("\033[1m"+'Topic', j, 'words:'+"\033[0m", end='  ')
  for k in range(10):
    print(output(j)[k][0], end=' ')
  print('                                                                                      ')

Topic 0 words:  close please this loan busy connect always no cheque ac                                                                                       
Topic 1 words:  drop end response no call customer so                                                                                          
Topic 2 words:  enquiryunsuccessful dropped callcall utan viche droppedf cenquiryunsuccessful categorycall her apurva                                                                                       
Topic 3 words:  receipt payments received payment not fuck updates massage must vandana                                                                                       
Topic 4 words:  slowness application technical in jhand bhopal slowly takes city pincode                                                                                       
Topic 5 words:  between conversation disconnect by call customer lost side long party                                                        

In [16]:
model.visualize_barchart()

In [17]:
model.visualize_topics()

In [18]:
# model.visualize_heatmap()

In [19]:
model1 = BERTopic(nr_topics=10)

In [23]:
docs = data.Cleaned_Desc.to_list()
 
topics, probabilities = model1.fit_transform(docs)

2023-05-23 09:19:03,392 - BERTopic - Transformed documents to Embeddings
2023-05-23 09:20:01,456 - BERTopic - Reduced dimensionality
2023-05-23 09:20:08,690 - BERTopic - Clustered reduced embeddings
2023-05-23 09:20:28,648 - BERTopic - Reduced number of topics from 523 to 10


In [24]:
model1.get_topic_freq()

,Topic,Count
0,0,20573
1,-1,3029
2,1,483
3,2,481
4,3,323
5,4,128
6,5,40
7,6,38
8,7,22
9,8,12


In [25]:
def output_1(x):
  return model1.get_topic(x)

In [26]:
for j in range(9):
  print("\033[1m"+'Topic', j, 'words:'+"\033[0m", end='  ')
  for k in range(10):
    print(output_1(j)[k][0], end=' ')
  print('                                                                                      ')

Topic 0 words:  customer to the call loan and not is no so                                                                                       
Topic 1 words:  receipt payments received payment not receipts cancel reconciled the noc                                                                                       
Topic 2 words:  registration unsuccessful mandate services login unable register authentication details to                                                                                       
Topic 3 words:  opening application technical not closed please open complete movement mil                                                                                       
Topic 4 words:  gh ph no                                                                                              
Topic 5 words:  headset issue call disconnected warning given response problem no cust                                                                                       
Topic 6 words

In [27]:
model1.visualize_barchart()

In [31]:
model1.visualize_topics()